In [1]:
!pip install transformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 182 kB 67.7 MB/s 
     |████████████████████████████████| 7.6 MB 40.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 5.0 MB/s 
     |████████████████████████████████| 84 kB 2.5 MB/s 
     |████████████████████████████████| 213 kB 75.0 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 84 kB 2.0 MB/s 
     |████████████████████████████████| 278 kB 64.2 MB/s 
     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 2.3 MB 26.9 MB/s 
     |████████████████████████████████| 106 kB 70.2 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
   

In [2]:
from transformers import AutoTokenizer, GPT2Tokenizer, GPT2ForSequenceClassification, GPT2Model

import torch 
from torch import nn

import numpy as np

import gradio as gr

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ECE1786 Project/Best model

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1fJllm6_aqmmawo8B6hq9rhewEDIBmSpR/ECE1786 Project/Best model


In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [4]:
class GPT2Classifier(nn.Module):
  def __init__(self, embedding_size, num_classes,max_seq_len, gpt_model_name):
    super(GPT2Classifier,self).__init__()
    self.gpt2 = GPT2Model.from_pretrained(gpt_model_name)
    # self.classifier_head = nn.Linear(embedding_size*max_seq_len, num_classes)    
    self.classifier_head = nn.Sequential(
                                nn.Linear(embedding_size*max_seq_len, 768),
                                nn.ReLU(),
                                nn.Dropout(0.1),
                                nn.Linear(768, num_classes)
                              )

      
  def forward(self, input_id, mask):
    gpt_out, _ = self.gpt2(input_ids=input_id, attention_mask=mask, return_dict=False)
    batch_size = gpt_out.shape[0]
    linear_output = self.classifier_head(gpt_out.view(batch_size,-1))
    return linear_output

In [13]:
model_name = "distilgpt2"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

song_genre_classifier = GPT2Classifier(embedding_size=768, num_classes=8, max_seq_len=256, gpt_model_name=model_name)
if use_cuda:
  song_genre_classifier = song_genre_classifier.cuda()

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
gpt_checkpoint_file = "distilgpt2-best-model.pt"
checkpoint = torch.load(gpt_checkpoint_file)
song_genre_classifier.load_state_dict(checkpoint)

<All keys matched successfully>

In [15]:
def process(text):
   text = text.replace("\n\n", "$$^^$$").replace("\n", " ").replace("$$^^$$", "\n")
   text = tokenizer(text, padding='max_length', max_length=256, truncation=True, return_tensors="pt")     
   return text

genre_labels = ['Blues', 'Country', 'EDM', 'Hip Hop', 'Indie', 'Pop', 'R&B', 'Rock']

def classifier(lyrics):
  #get input
  input = process(lyrics)
  mask = input['attention_mask'].to(device)
  input_id = input['input_ids'].squeeze(1).to(device)

  #predict
  output = song_genre_classifier(input_id, mask)
  pred = output.argmax(dim=1)
  genre = genre_labels[pred]
  return genre

In [16]:
#GRADIO PART
def run_demo():
  input_box = gr.Textbox(label="Enter Song Lyrics to Predict", lines=10, placeholder="Sentence needs to be longer than 3 words...")
  prediction_box = gr.Textbox(label="Genre")

  demo = gr.Interface(
      fn=classifier,
      inputs=input_box,
      outputs=[prediction_box],
  )
  demo.launch(debug=True)

  

In [ ]:
run_demo()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>